In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [ ]:
# ---------------------------
# Dataset Path
# ---------------------------
dataset_path = "/kaggle/input/fer2013"

# Image size for FER2013
img_size = (48,48)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(
    dataset_path + "/train",
    target_size=img_size,
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=10,
    shuffle=True
)

X_small, y_small = next(train_data)

In [ ]:

# ---------------------------
# DNN Model
# ---------------------------
dnn = Sequential([
    Flatten(input_shape=(48,48,1)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(train_data.class_indices), activation='softmax')
])

dnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
dnn.fit(X_small, y_small, epochs=10, verbose=0)

In [ ]:
# ---------------------------
# CNN Model
# ---------------------------
cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_data.class_indices), activation='softmax')
])

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn.fit(X_small, y_small, epochs=10, verbose=0)

In [ ]:
# ---------------------------
# Pick a random sample
# ---------------------------
idx = random.randint(0, len(X_small)-1)
sample = X_small[idx].reshape(1,48,48,1)
true_label = np.argmax(y_small[idx])

pred_dnn = np.argmax(dnn.predict(sample))
pred_cnn = np.argmax(cnn.predict(sample))

# Emotion classes from dataset
emotion_dict = {v: k for k,v in train_data.class_indices.items()}

In [ ]:
# ---------------------------
# Display output
# ---------------------------
plt.imshow(X_small[idx].reshape(48,48), cmap='gray')
plt.title(f"True: {emotion_dict[true_label]}\nDNN: {emotion_dict[pred_dnn]} | CNN: {emotion_dict[pred_cnn]}")
plt.axis('off')
plt.show()

In [ ]:
dnn.save('emotion_dnn_model.h5')
cnn.save('emotion_cnn_model.h5')

In [ ]:
print("DNN Performance:")
print("Accuracy:", accuracy_score(y_true, y_pred_dnn_classes))
print("Precision:", precision_score(y_true, y_pred_dnn_classes, average='weighted'))
print("Recall:", recall_score(y_true, y_pred_dnn_classes, average='weighted'))
print("F1-score:", f1_score(y_true, y_pred_dnn_classes, average='weighted'))
print(classification_report(y_true, y_pred_dnn_classes, target_names=class_labels))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# ---------------------------
# Load a validation set
# ---------------------------
val_data = datagen.flow_from_directory(
    dataset_path + "/validation",
    target_size=img_size,
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=32,
    shuffle=False
)

# Get true labels
y_true = val_data.classes  # integer labels
class_labels = list(val_data.class_indices.keys())

# ---------------------------
# Evaluate DNN
# ---------------------------
y_pred_dnn = dnn.predict(val_data)
y_pred_dnn_classes = np.argmax(y_pred_dnn, axis=1)

print("DNN Performance:")
print("Accuracy:", accuracy_score(y_true, y_pred_dnn_classes))
print("Precision:", precision_score(y_true, y_pred_dnn_classes, average='weighted'))
print("Recall:", recall_score(y_true, y_pred_dnn_classes, average='weighted'))
print("F1-score:", f1_score(y_true, y_pred_dnn_classes, average='weighted'))
print(classification_report(y_true, y_pred_dnn_classes, target_names=class_labels))

# ---------------------------
# Evaluate CNN
# ---------------------------
y_pred_cnn = cnn.predict(val_data)
y_pred_cnn_classes = np.argmax(y_pred_cnn, axis=1)

print("CNN Performance:")
print("Accuracy:", accuracy_score(y_true, y_pred_cnn_classes))
print("Precision:", precision_score(y_true, y_pred_cnn_classes, average='weighted'))
print("Recall:", recall_score(y_true, y_pred_cnn_classes, average='weighted'))
print("F1-score:", f1_score(y_true, y_pred_cnn_classes, average='weighted'))
print(classification_report(y_true, y_pred_cnn_classes, target_names=class_labels))

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# ---------------------------
# Load test set instead of validation
# ---------------------------
test_data = datagen.flow_from_directory(
    dataset_path + "/test",
    target_size=img_size,
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=32,
    shuffle=False
)

# Get true labels
y_true = test_data.classes
class_labels = list(test_data.class_indices.keys())

# ---------------------------
# Evaluate DNN
# ---------------------------
y_pred_dnn = dnn.predict(test_data)
y_pred_dnn_classes = np.argmax(y_pred_dnn, axis=1)

print("📊 DNN Performance")
print("Accuracy:", accuracy_score(y_true, y_pred_dnn_classes))
print("Precision:", precision_score(y_true, y_pred_dnn_classes, average='weighted'))
print("Recall:", recall_score(y_true, y_pred_dnn_classes, average='weighted'))
print("F1-score:", f1_score(y_true, y_pred_dnn_classes, average='weighted'))
print(classification_report(y_true, y_pred_dnn_classes, target_names=class_labels))

# ---------------------------
# Evaluate CNN
# ---------------------------
y_pred_cnn = cnn.predict(test_data)
y_pred_cnn_classes = np.argmax(y_pred_cnn, axis=1)

print("📊 CNN Performance")
print("Accuracy:", accuracy_score(y_true, y_pred_cnn_classes))
print("Precision:", precision_score(y_true, y_pred_cnn_classes, average='weighted'))
print("Recall:", recall_score(y_true, y_pred_cnn_classes, average='weighted'))
print("F1-score:", f1_score(y_true, y_pred_cnn_classes, average='weighted'))
print(classification_report(y_true, y_pred_cnn_classes, target_names=class_labels))

Found 7178 images belonging to 7 classes.
  1/225 ━━━━━━━━━━━━━━━━━━━━ 13s 59ms/step

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


225/225 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step
📊 DNN Performance
Accuracy: 0.19281136806910001
Precision: 0.16879150442026647
Recall: 0.19281136806910001
F1-score: 0.16136609900966792
              precision    recall  f1-score   support

       angry       0.15      0.22      0.18       958
     disgust       0.00      0.00      0.00       111
        fear       0.15      0.44      0.22      1024
       happy       0.30      0.33      0.31      1774
     neutral       0.00      0.00      0.00      1233
         sad       0.23      0.08      0.12      1247
    surprise       0.13      0.06      0.08       831

    accuracy                           0.19      7178
   macro avg       0.14      0.16      0.13      7178
weighted avg       0.17      0.19      0.16      7178

  1/225 ━━━━━━━━━━━━━━━━━━━━ 14s 64ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:13

225/225 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step
📊 CNN Performance
Accuracy: 0.24895514070771801
Precision: 0.12344522274034203
Recall: 0.24895514070771801
F1-score: 0.11256324471831332
              precision    recall  f1-score   support

       angry       0.00      0.00      0.00       958
     disgust       0.00      0.00      0.00       111
        fear       0.17      0.00      0.01      1024
       happy       0.25      0.97      0.40      1774
     neutral       0.00      0.00      0.00      1233
         sad       0.22      0.04      0.07      1247
    surprise       0.00      0.00      0.00       831

    accuracy                           0.25      7178
   macro avg       0.09      0.15      0.07      7178
weighted avg       0.12      0.25      0.11      7178



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:13